In [1]:
from datasets import load_dataset
import json

# Load dataset from HF
ds = load_dataset("HackHedron/English_Telugu_Parallel_Corpus")

# Convert to Python list
records = [dict(row) for row in ds['train']]

# Save JSON
with open("source_target.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

print("Saved as source_target.json")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/164 [00:00<?, ?B/s]

English-Telugu Parallel Corpus Dataset.c(…):   0%|          | 0.00/124M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/433845 [00:00<?, ? examples/s]

Saved as source_target.json


In [2]:
# --- Connect to Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Import libraries ---
from datasets import load_dataset
import json
import os

# --- Load dataset from Hugging Face ---
ds = load_dataset("HackHedron/English_Telugu_Parallel_Corpus")

# --- Convert dataset to list of dictionaries ---
records = [dict(row) for row in ds['train']]

# --- Output path inside Google Drive ---
output_path = "/content/drive/MyDrive/source_target.json"

# --- Save JSON to Google Drive ---
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=4)

print("Saved JSON file to:", output_path)

Mounted at /content/drive
Saved JSON file to: /content/drive/MyDrive/source_target.json


In [3]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Choose your model (pick one)
model_name = "google/flan-t5-small"      # ~300 MB  → ultra fast on CPU
# model_name = "google/flan-t5-base"     # ~990 MB  → best balance (recommended)
# model_name = "google/flan-t5-large"    # ~3 GB   → very strong
# model_name = "google/flan-t5-xl"          # ~11 GB  → beast mode (needs RAM)

# Where to save it forever in your Drive
save_directory = f"/content/drive/MyDrive/{model_name.replace('/', '_')}"

os.makedirs(save_directory, exist_ok=True)

print(f"Downloading {model_name}...")
print("This may take 1–10 minutes depending on size...")

# Download tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Download model
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name,
    torch_dtype="auto",           # auto-detects best (bfloat16/float16/float32)
    low_cpu_mem_usage=True       # helps with RAM on CPU
)

# Save to Drive
print(f"Saving tokenizer + model to {save_directory}...")
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)

print("SUCCESS! FLAN-T5 is now saved in your Google Drive")
print(f"Location: {save_directory}")
print("You can now load it anytime with:")
print(f"   tokenizer = AutoTokenizer.from_pretrained('{save_directory}')")
print(f"   model = AutoModelForSeq2SeqLM.from_pretrained('{save_directory}')")

This may take 1–10 minutes depending on size...


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Saving tokenizer + model to /content/drive/MyDrive/google_flan-t5-small...
SUCCESS! FLAN-T5 is now saved in your Google Drive
Location: /content/drive/MyDrive/google_flan-t5-small
You can now load it anytime with:
   tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/google_flan-t5-small')
   model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/google_flan-t5-small')


In [4]:
# ======================= 0. INSTALL =======================
!pip install -q transformers datasets peft accelerate sentencepiece torch

# ======================= 1. MOUNT DRIVE ===================
from google.colab import drive
drive.mount('/content/drive')

# ======================= 2. CONFIG ========================
import os, json, torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, get_peft_model

model_type = "flan"
hf_model_id = "google/flan-t5-base"

model_dir  = "/content/drive/MyDrive/google_flan-t5-small"
data_path  = "/content/drive/MyDrive/source_target.json"
output_dir = "/content/drive/MyDrive/Major Project"

os.makedirs(model_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)

# ======================= 3. AUTO DOWNLOAD =================
if not os.path.exists(os.path.join(model_dir, "config.json")):
    print("⬇️ Model not found → Downloading from Hugging Face...")

    tokenizer = AutoTokenizer.from_pretrained(hf_model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(hf_model_id)

    tokenizer.save_pretrained(model_dir)
    model.save_pretrained(model_dir)

    print("✅ Model downloaded & saved to Drive")
else:
    print("✅ Model found in Drive → Loading locally")

    os.environ["HF_HUB_DISABLE"] = "1"
    os.environ["TRANSFORMERS_OFFLINE"] = "1"

    tokenizer = AutoTokenizer.from_pretrained(model_dir, local_files_only=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_dir, local_files_only=True)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# ======================= 4. LOAD DATA =====================
print("\n📂 Loading dataset...")
data = []

if data_path.endswith(".jsonl"):
    with open(data_path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
else:
    with open(data_path, "r", encoding="utf-8") as f:
        data = json.load(f)

print(f"✅ Loaded {len(data):,} examples")

# ======================= 5. AUTO FORMAT ===================
def format_example(x):
    if isinstance(x, dict):
        if "source" in x and "target" in x:
            return {"input_text": x["source"], "target_text": x["target"]}
        if "question" in x and "answer" in x:
            return {"input_text": x["question"], "target_text": x["answer"]}
        if "text" in x:
            return {"input_text": x["text"], "target_text": x["text"]}
    return {"input_text": str(x), "target_text": str(x)}

dataset = Dataset.from_list([format_example(x) for x in data])

# ======================= 6. SHUFFLE + LIMIT ===============
dataset = dataset.shuffle(seed=42)
print("🔀 Dataset shuffled")

# Limit to 12,000 samples
dataset = dataset.select(range(12000))
print("🎯 Using only first 12,000 samples")

# ======================= 7. TOKENIZATION ==================
def tokenize(ex):
    inp = tokenizer(
        ex["input_text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    lab = tokenizer(
        ex["target_text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    inp["labels"] = lab["input_ids"]
    return inp

tokenized = dataset.map(tokenize, batched=True, remove_columns=dataset.column_names)
print("Total training samples:", len(tokenized))

# Split into training and testing sets
train_test_split = tokenized.train_test_split(test_size=0.1, seed=42)
train_ds = train_test_split["train"]
test_ds = train_test_split["test"]

print(f"Training samples: {len(train_ds)}, Test samples: {len(test_ds)}")

# ======================= 8. APPLY LoRA ====================
lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)

# ======================= 9. TRAIN =========================
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    warmup_steps=100,                 # 🔥 More stable training
    logging_steps=20,
    save_steps=1000,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    tokenizer=tokenizer
)

print("\n🚀 Training Started...\n")
trainer.train()

# ======================= 10. SAVE FINAL MODEL ==============
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("\n✅✅✅ FINETUNING COMPLETE (2 EPOCHS, 12K SAMPLES) ✅✅✅")
print("📦 Final model saved at:")
print(output_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Model found in Drive → Loading locally

📂 Loading dataset...
✅ Loaded 433,845 examples
🔀 Dataset shuffled
🎯 Using only first 12,000 samples


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Total training samples: 12000
Training samples: 10800, Test samples: 1200


/tmp/ipython-input-104130091.py:141: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



🚀 Training Started...



Step,Training Loss
20,42.681300
40,42.529600
60,39.903000
80,34.716100
100,26.791300
120,15.055800
140,7.214300
160,5.281700
180,4.417500
200,4.015000



✅✅✅ FINETUNING COMPLETE (2 EPOCHS, 12K SAMPLES) ✅✅✅
📦 Final model saved at:
/content/drive/MyDrive/Major Project


In [5]:
# ========= TRANSLATION ACCURACY (BLEU) =========
!pip install -q evaluate sacrebleu
import torch
import evaluate

bleu = evaluate.load("sacrebleu")

def compute_bleu_score(model, tokenizer, dataset, max_samples=200):
    predictions, references = [], []

    model.eval()
    for i, ex in enumerate(dataset):
        if i >= max_samples:
            break

        input_ids = torch.tensor(ex["input_ids"]).unsqueeze(0).to(model.device)
        attention_mask = torch.tensor(ex["attention_mask"]).unsqueeze(0).to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=128
            )

        pred = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        ref  = tokenizer.decode(ex["labels"], skip_special_tokens=True)

        predictions.append(pred)
        references.append([ref])

    print(f"Preds length: {len(predictions)}")
    print(f"Refs length: {len(references)}")
    result = bleu.compute(predictions=predictions, references=references)
    print(f"BLEU result: {result}")
    return result["score"]


# ▶ RUN ACCURACY
bleu_score = compute_bleu_score(model, tokenizer, test_ds)

print(f"📊 Translation Accuracy (BLEU Score): {round(bleu_score, 2)}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.1 MB/s eta 0:00:00


Preds length: 200
Refs length: 200
BLEU result: {'score': 96.41797498129152, 'counts': [4679, 4459, 4232, 4022], 'totals': [4723, 4523, 4323, 4123], 'precisions': [99.0683887359729, 98.5850099491488, 97.89498033772843, 97.55032743148193], 'bp': 0.9811247268441933, 'sys_len': 4723, 'ref_len': 4813}
📊 Translation Accuracy (BLEU Score): 96.42


In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ================= LOAD MODEL =================
model_name = "facebook/nllb-200-distilled-600M"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# ================= SET LANGUAGE DIRECTION =================
tokenizer.src_lang = "tel_Telu"                 # Telugu source
tgt_lang = "eng_Latn"                            # English target
tgt_lang_id = tokenizer.convert_tokens_to_ids(tgt_lang)

# ================= TRANSLATION FUNCTION =================
def translate_telugu_to_english(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            forced_bos_token_id=tgt_lang_id,
            max_new_tokens=64,
            num_beams=4
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# ================= TEST =================
telugu_text = "నాకు చాలా నొప్పిగా ఉంది"
english_output = translate_telugu_to_english(telugu_text)

print("TELUGU :", telugu_text)
print("ENGLISH:", english_output)

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

TELUGU : నాకు చాలా నొప్పిగా ఉంది
ENGLISH: I'm in a lot of pain.


In [7]:
telugu_text = "నాకు జ్వరంగా ఉంది"
english_output = translate_telugu_to_english(telugu_text)

print("TELUGU :", telugu_text)
print("ENGLISH:", english_output)

TELUGU : నాకు జ్వరంగా ఉంది
ENGLISH: I have a fever.


In [8]:
telugu_text = "అతనికి తలనొప్పి ఉంది"
english_output = translate_telugu_to_english(telugu_text)

print("TELUGU :", telugu_text)
print("ENGLISH:", english_output)

TELUGU : అతనికి తలనొప్పి ఉంది
ENGLISH: He has a headache.


In [9]:
telugu_text = "ఆమె చాలా బాధలో ఉంది"
english_output = translate_telugu_to_english(telugu_text)

print("TELUGU :", telugu_text)
print("ENGLISH:", english_output)

TELUGU : ఆమె చాలా బాధలో ఉంది
ENGLISH: She's in a lot of pain.
